Import Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

#!pip install rapidfuzz
from rapidfuzz import process, fuzz


Import Data

In [2]:
# Synthetic Company Data
df_company = pd.read_csv(r'Realistic_Synthetic_Bank_Customers.csv')
print('Synthetic Bank Customer Data:\n',df_company.head())

print('\n')

# SDN List Data
df_sdn = pd.read_csv('SDN_OFAC_Data.csv')
print('SDN List Data:\n',df_sdn.head())


Synthetic Bank Customer Data:
           ID                             Name Customer Type  \
0  CUST00001  Rodriguez, Figueroa and Sanchez        Entity   
1  CUST00002                        Doyle Ltd        Entity   
2  CUST00003                   Kimberly Garza    Individual   
3  CUST00004                     Brandon Hall    Individual   
4  CUST00005                 Kristina Baldwin    Individual   

  Organization (Country)  
0                 Mexico  
1          United States  
2                Germany  
3                 Mexico  
4          United States  


SDN List Data:
     ID                       Name  Type Organization
0   36     AEROCARIBBEAN AIRLINES  -0-          CUBA
1  173  ANGLO-CARIBBEAN CO., LTD.  -0-          CUBA
2  306     BANCO NACIONAL DE CUBA  -0-          CUBA
3  424         BOUTIQUE LA MAISON  -0-          CUBA
4  475               CASA DE CUBA  -0-          CUBA


Obtain Summary Stats on Data

In [3]:
# Synthetic Company Data
print('Synthetic Bank Customer Data Statistics:\n',df_company.describe())

print('\n')

# SDN List Data
print('SDN List Data Statistics:\n',df_sdn.describe())


Synthetic Bank Customer Data Statistics:
                ID           Name Customer Type Organization (Country)
count        1000           1000          1000                   1000
unique       1000            997             2                      6
top     CUST00001  Jessica Smith    Individual          United States
freq            1              2           716                    406


SDN List Data Statistics:
            ID                Name   Type    Organization
count   17563               17562  17562           17562
unique  17563               17531      4             227
top        36  AL-AQSA FOUNDATION   -0-   RUSSIA-EO14024
freq        1                   4   8844            5735


Identify and Remove Nulls

In [4]:
# Synthetic Company Data
print('Synthetic Bank Customer Data:\n',df_company.isnull().sum())

print('\n')

# SDN List Data
print('SDN List Data:\n',df_sdn.isnull().sum())

print('\n')

# Identify null values
null_values = df_sdn[df_sdn.isnull().any(axis=1)]
print(null_values)

# There is 1 instance of missing values. Removed null row since it is a data quality issue
df_sdn = df_sdn.dropna()


Synthetic Bank Customer Data:
 ID                        0
Name                      0
Customer Type             0
Organization (Country)    0
dtype: int64


SDN List Data:
 ID              0
Name            1
Type            1
Organization    1
dtype: int64


      ID Name Type Organization
17562    NaN  NaN          NaN


Preprocessing - Clean Names

In [5]:
def normalize_name(name):
    return (
        str(name)
        .lower()
        .replace('.', '')
        .replace(',', '')
        .replace('-', ' ')
        .replace('  ', ' ')
        .strip()
    )

df_company['Clean_Name'] = df_company['Name'].apply(normalize_name)
df_sdn['Clean_Name'] = df_sdn['Name'].apply(normalize_name)

#Example
print(df_sdn)


          ID                       Name        Type Organization  \
0         36     AEROCARIBBEAN AIRLINES        -0-          CUBA   
1        173  ANGLO-CARIBBEAN CO., LTD.        -0-          CUBA   
2        306     BANCO NACIONAL DE CUBA        -0-          CUBA   
3        424         BOUTIQUE LA MAISON        -0-          CUBA   
4        475               CASA DE CUBA        -0-          CUBA   
...      ...                        ...         ...          ...   
17557  54624        QUEENS RING LIMITED        -0-   SDGT] [IFSR   
17558  54625        RADIX TRADE LIMITED        -0-   SDGT] [IFSR   
17559  54626     STAR OILGLOBAL LIMITED        -0-   SDGT] [IFSR   
17560  54627           MARMERTH LIMITED        -0-   SDGT] [IFSR   
17561  54707  ALBANESE, Francesca Paola  individual  ICC-EO14203   

                     Clean_Name  
0        aerocaribbean airlines  
1        anglo caribbean co ltd  
2        banco nacional de cuba  
3            boutique la maison  
4            

Obtain data similar in both Datasets

In [6]:
# Get raw similarity matrix
scores = process.cdist(
    df_company['Clean_Name'],
    df_sdn['Clean_Name'],
    scorer=fuzz.token_sort_ratio
)

# Filter by threshold and flatten results
matches = []
threshold = 90

for i in range(scores.shape[0]): # Loop through each row 
    for j in range(scores.shape[1]): # Loop through each column 
        score = scores[i, j]
        if score >= threshold:
            matches.append({
                'Bank_Name': df_company.iloc[i]['Name'],
                'OFAC_Name': df_sdn.iloc[j]['Name'],
                'Score': score
            })

# Return results with a threshold value of at least 90
df_matches = pd.DataFrame(matches)
df_matches

,Bank_Name,OFAC_Name,Score
0,Jim Newton,NEWTON ITM,90.0
1,Milan Ivanovic,"IVANOVIC, Milan",100.0


Refer to the document "Report" to view the report for this notebook.